In [2]:
import json
import re
import requests
import time
import sys
from pathlib import Path
from typing import List, Dict, Optional, Tuple
from dataclasses import dataclass
from datetime import datetime
import pandas as pd

@dataclass
class PipelineConfig:
    """流水线配置"""
    data_path: Path
    output_base: Path
    lm_studio_url: str = "http://localhost:1234/v1/chat/completions"
    model_name: str = "Qwen3-8B-Q4_K_M.gguf"
    max_works: Optional[int] = None  
    test_mode: bool = False  

class TripleExtractor:
    """三元组提取器"""
    
    def __init__(self, config: PipelineConfig):
        self.config = config
        self.extractor_type = "triples"
        self.output_dir = config.output_base / self.extractor_type
        self.output_dir.mkdir(parents=True, exist_ok=True)
        
        # 角色别名词典
        self.character_aliases = {
            "刘彻": ["刘彻", "陛下", "皇帝", "孤", "朕", "帝王", "天子", "武帝", "皇上", "彻"],
            "卫青": ["卫青", "卫将军", "仲卿", "姐夫", "青", "卫大人", "将军", "仲青", "卫帅"]
        }
        
        # 创建反向映射
        self.alias_to_main = {}
        for main_char, aliases in self.character_aliases.items():
            for alias in aliases:
                self.alias_to_main[alias.lower()] = main_char
    
    def normalize_character(self, text: str) -> Optional[str]:
        """标准化角色名"""
        if not text:
            return None
        
        text_lower = text.strip().lower()
        
        for alias, main_char in self.alias_to_main.items():
            if alias == text_lower:
                return main_char
        
        for alias, main_char in self.alias_to_main.items():
            if alias in text_lower:
                return main_char
        
        return None
    
    def test_api_connection(self) -> bool:
        """测试API连接"""
        print(f"测试三元组提取 API连接...")
        
        headers = {"Content-Type": "application/json"}
        payload = {
            "model": self.config.model_name,
            "messages": [{"role": "user", "content": "请回复'连接成功'"}],
            "temperature": 0.1,
            "max_tokens": 20
        }
        
        try:
            response = requests.post(self.config.lm_studio_url, json=payload, headers=headers, timeout=30)
            if response.status_code == 200:
                result = response.json()
                if "choices" in result and len(result["choices"]) > 0:
                    content = result["choices"][0]["message"]["content"].strip()
                    print(f"✅ 三元组提取 API连接成功: {content}")
                    return True
            else:
                print(f"❌ 三元组提取 API连接失败: {response.status_code}")
                return False
        except Exception as e:
            print(f"❌ 三元组提取 API连接异常: {e}")
            return False
    
    def call_llama_api(self, prompt: str, max_tokens: int = 500, temperature: float = 0.05) -> str:
        """调用API"""
        headers = {"Content-Type": "application/json"}
        
        payload = {
            "model": self.config.model_name,
            "messages": [
                {
                    "role": "system", 
                    "content": f"你是一个准确的三元组提取助手。请直接输出结果，不要包含任何思考过程或额外解释。"
                },
                {"role": "user", "content": prompt}
            ],
            "temperature": temperature,
            "max_tokens": max_tokens,
            "top_p": 0.9,
            "stream": False
        }
        
        try:
            response = requests.post(self.config.lm_studio_url, 
                                   json=payload, 
                                   headers=headers, 
                                   timeout=120)
            response.raise_for_status()
            result = response.json()
            
            if "choices" in result and len(result["choices"]) > 0:
                content = result["choices"][0]["message"]["content"].strip()
                return self.clean_llm_output(content)
                
        except Exception as e:
            print(f"  API调用错误: {e}")
        
        return ""
    
    def clean_llm_output(self, text: str) -> str:
        """清理LLM输出"""
        patterns_to_remove = [
            r'<think>.*?</think>',
            r'首先[，,]?.*?然后[，,]?',
            r'分析如下[：:]?',
            r'思考[：:]?',
            r'我需要.*?提取',
            r'根据.*?文本',
            r'综上所述',
            r'因此，',
            r'所以，'
        ]
        
        cleaned = text
        for pattern in patterns_to_remove:
            cleaned = re.sub(pattern, '', cleaned, flags=re.DOTALL)
        
        lines = [line.strip() for line in cleaned.split('\n') if line.strip()]
        return '\n'.join(lines)
    
    def split_text_by_dialogue(self, text: str, max_chunk_size: int = 600) -> List[Dict]:
        """按对话和段落分割文本"""
        if not text:
            return []
        
        if len(text) <= max_chunk_size:
            return [{"text": text, "start": 0, "end": len(text)}]
        
        paragraphs = text.split('\n')
        
        chunks = []
        current_chunk = ""
        current_start = 0
        char_count = 0
        
        for para in paragraphs:
            para = para.strip()
            if not para:
                char_count += 1  
                continue
            
            para_with_newline = para + "\n"
            para_length = len(para_with_newline)
            
            if len(current_chunk) + len(para) + 1 <= max_chunk_size:
                if current_chunk:
                    current_chunk += "\n" + para
                else:
                    current_chunk = para
                    current_start = char_count
                
                char_count += para_length
            else:
                
                if current_chunk:
                    chunks.append({
                        "text": current_chunk,
                        "start": current_start,
                        "end": char_count
                    })

                if len(para) > max_chunk_size:
                    sentences = re.split(r'([。！？!?])', para)
                    temp_sentence = ""
                    temp_start = char_count
                    
                    for i in range(0, len(sentences), 2):
                        if i < len(sentences):
                            sentence = sentences[i]
                            if i+1 < len(sentences):
                                sentence += sentences[i+1]
                            
                            sentence_length = len(sentence)
                            
                            if len(temp_sentence) + len(sentence) <= max_chunk_size:
                                if temp_sentence:
                                    temp_sentence += sentence
                                else:
                                    temp_sentence = sentence
                                    temp_start = char_count
                            else:
                                if temp_sentence:
                                    chunks.append({
                                        "text": temp_sentence,
                                        "start": temp_start,
                                        "end": char_count
                                    })
                                temp_sentence = sentence
                                temp_start = char_count
                            
                            char_count += sentence_length
                    
                    if temp_sentence:
                        chunks.append({
                            "text": temp_sentence,
                            "start": temp_start,
                            "end": char_count
                        })
                else:
                    current_chunk = para
                    current_start = char_count
                    char_count += para_length
        
        if current_chunk:
            chunks.append({
                "text": current_chunk,
                "start": current_start,
                "end": len(text)
            })
        
        return chunks
    
    def create_prompt(self, text: str, chunk_id: int = 0) -> str:
        """创建三元组抽取提示词"""
        prompt = f"""你是一个专门分析中文历史同人小说中人物互动的专家。请仔细分析以下文本，找出所有刘彻和卫青之间的直接互动。

文本内容：
{text}

提取要求：
1. 只提取刘彻和卫青之间的直接互动，忽略其他角色
2. 互动类型包括但不限于：
   - 言语互动：说、问、回答、命令、指示、请求、劝说、调笑、责骂等
   - 身体互动：触摸、亲吻、拥抱、示意、注视、推搡、抚摸、咬等
   - 情感互动：安慰、安抚、威胁、警告、鼓励等
   - 性互动：性接触、性行为等
3. 每个互动必须是直接的，有明确的主体、动作和客体
4. 动作词要具体，反映原文的语义
5. 对于每个提取的三元组，请同时提供原文中相关的句子或短语作为证据

输出格式：
每行一个三元组，格式为：[主体] [动作] [客体] || 原始文本片段
主体和客体必须是"刘彻"或"卫青"
原始文本片段应该是提取该互动的具体原文部分的完整句子。请仔细阅读文本，提取刘彻和卫青之间的互动关系。尽量细致，对于每一个动词，互动，都要思考是否有提取的可能性。例如"刘彻当下就赶他走"，"赶走"就是一个动词。如果有非常直接露骨的具体描写，则允许采用中性词"性接触"标注。

示例：
[刘彻] [命令] [卫青] || 刘彻对卫青说："去做这件事。"
[卫青] [回答] [刘彻] || 卫青恭敬地回答："遵命，陛下。"

如果没有相关互动，请输出：无相关互动

现在请分析："""
        
        return prompt
    
    def find_source_snippet(self, full_text: str, triple: Dict, chunk_info: Dict) -> str:
        """在原文中查找三元组对应的片段"""
        try:
            # 查找主体和客体在文本中的位置
            subject_keywords = []
            object_keywords = []
            
            if triple["subject_normalized"] == "刘彻":
                subject_keywords = ["刘彻", "陛下", "皇帝", "朕", "帝王", "天子", "武帝", "皇上", "彻"]
            else:
                subject_keywords = ["卫青", "卫将军", "仲卿", "将军", "青", "卫大人"]
            
            if triple["object_normalized"] == "刘彻":
                object_keywords = ["刘彻", "陛下", "皇帝", "朕", "帝王", "天子", "武帝", "皇上", "彻"]
            else:
                object_keywords = ["卫青", "卫将军", "仲卿", "将军", "青", "卫大人"]
            
            chunk_text = chunk_info["text"]
            sentences = re.split(r'[。！？!?]', chunk_text)
            
            for sentence in sentences:
                sentence = sentence.strip()
                if not sentence:
                    continue
                
                has_subject = any(keyword in sentence for keyword in subject_keywords)
                has_object = any(keyword in sentence for keyword in object_keywords)
                
                if has_subject and has_object:
                    action_verbs = ["说", "问", "命令", "示意", "吻", "咬", "抚摸", "注视", "调笑", "责骂", "安慰"]
                    for verb in action_verbs:
                        if verb in sentence and verb in triple["action"]:
                            return sentence + "。"
                    
                    return sentence + "。"
            
            # 如果没有找到完整句子
            for keyword in subject_keywords:
                if keyword in chunk_text:
                    idx = chunk_text.find(keyword)
                    start = max(0, idx - 50)
                    end = min(len(chunk_text), idx + 100)
                    return chunk_text[start:end] + "..."
            
        except Exception as e:
            print(f"  查找源片段时出错: {e}")
        
        return ""
    
    def parse_response(self, response: str, full_text: str, chunk_info: Dict) -> List[Dict]:
        """解析三元组响应"""
        if not response or "无相关互动" in response:
            return []
        
        triples = []
        lines = response.strip().split('\n')
        
        for line in lines:
            line = line.strip()
            if not line:
                continue
            
            pattern = r'\[([^\]]+)\]\s+\[([^\]]+)\]\s+\[([^\]]+)\]\s*\|\|\s*(.+)'
            match = re.search(pattern, line)
            
            if match:
                subject = match.group(1).strip()
                action = match.group(2).strip()
                object_ = match.group(3).strip()
                source_snippet = match.group(4).strip() if len(match.groups()) > 3 else ""
                
                norm_subject = self.normalize_character(subject)
                norm_object = self.normalize_character(object_)
                
                if norm_subject in ["刘彻", "卫青"] and norm_object in ["刘彻", "卫青"]:
                    direction = "刘彻→卫青" if norm_subject == "刘彻" and norm_object == "卫青" else "卫青→刘彻"
                    
                    if not source_snippet:
                        source_snippet = self.find_source_snippet(full_text, {
                            "subject_normalized": norm_subject,
                            "action": action,
                            "object_normalized": norm_object
                        }, chunk_info)
                    
                    triples.append({
                        "subject_raw": subject,
                        "subject_normalized": norm_subject,
                        "action": action,
                        "object_raw": object_,
                        "object_normalized": norm_object,
                        "direction": direction,
                        "source_snippet": source_snippet,
                        "raw_triple": line,
                        "chunk_start": chunk_info.get("start", 0),
                        "chunk_end": chunk_info.get("end", 0)
                    })
            else:
                old_pattern = r'\[([^\]]+)\]\s+\[([^\]]+)\]\s+\[([^\]]+)\]'
                old_match = re.search(old_pattern, line)
                
                if old_match:
                    subject = old_match.group(1).strip()
                    action = old_match.group(2).strip()
                    object_ = old_match.group(3).strip()
                    
                    norm_subject = self.normalize_character(subject)
                    norm_object = self.normalize_character(object_)
                    
                    if norm_subject in ["刘彻", "卫青"] and norm_object in ["刘彻", "卫青"]:
                        direction = "刘彻→卫青" if norm_subject == "刘彻" and norm_object == "卫青" else "卫青→刘彻"
                        
                        source_snippet = self.find_source_snippet(full_text, {
                            "subject_normalized": norm_subject,
                            "action": action,
                            "object_normalized": norm_object
                        }, chunk_info)
                        
                        triples.append({
                            "subject_raw": subject,
                            "subject_normalized": norm_subject,
                            "action": action,
                            "object_raw": object_,
                            "object_normalized": norm_object,
                            "direction": direction,
                            "source_snippet": source_snippet,
                            "raw_triple": line,
                            "chunk_start": chunk_info.get("start", 0),
                            "chunk_end": chunk_info.get("end", 0)
                        })
        
        return triples
    
    def extract_from_work(self, work: Dict) -> Dict:
        """从单个作品中抽取三元组"""
        work_id = work.get('work_id')
        title = work.get('title', '无标题')
        work_text = work.get('work_text', '')
        
        print(f"\n📚 处理作品 {work_id}: {title}")
        
        if not work_text:
            print("⚠️  无文本内容，跳过")
            return None
        
        # 计算updated_words（字数）
        updated_words = len(work_text.strip())
        print(f"   文本字数: {updated_words} 字符")
        
        chunks = self.split_text_by_dialogue(work_text)
        all_triples = []
        start_time = time.time()
        
        for i, chunk_info in enumerate(chunks, 1):
            chunk_text = chunk_info["text"]
            print(f"   处理块 {i}/{len(chunks)} (长度: {len(chunk_text)} 字符)")
            
            prompt = self.create_prompt(chunk_text, i)
            response = self.call_llama_api(prompt)
            
            if response:
                triples = self.parse_response(response, work_text, chunk_info)
                if triples:
                    all_triples.extend(triples)
                    print(f"     提取到 {len(triples)} 个三元组")
            
            if i < len(chunks):
                time.sleep(1)
        
        # 去重
        seen = set()
        final_triples = []
        for triple in all_triples:
            key = (triple["subject_normalized"], triple["action"], triple["object_normalized"], triple["source_snippet"][:100])
            if key not in seen:
                seen.add(key)
                final_triples.append(triple)
        
        elapsed_time = time.time() - start_time
        
        # 统计信息
        liuche_to_weiqing = len([t for t in final_triples if t["direction"] == "刘彻→卫青"])
        weiqing_to_liuche = len([t for t in final_triples if t["direction"] == "卫青→刘彻"])
        total_triples = len(final_triples)
        
        result = {
            "work_id": work_id,
            "title": title,
            "author": work.get('author', ''),
            "original_words": work.get('words', 0),
            "updated_words": updated_words,
            "total_triples": total_triples,
            "extraction_time_seconds": round(elapsed_time, 2),
            "triples": final_triples,
            "statistics": {
                "liuche_to_weiqing": liuche_to_weiqing,
                "weiqing_to_liuche": weiqing_to_liuche,
                "liuche_to_weiqing_percentage": round(liuche_to_weiqing / total_triples * 100, 2) if total_triples > 0 else 0,
                "weiqing_to_liuche_percentage": round(weiqing_to_liuche / total_triples * 100, 2) if total_triples > 0 else 0,
                "with_source_snippet": len([t for t in final_triples if t.get("source_snippet", "")]),
                "unique_actions": len(set([t["action"] for t in final_triples]))
            }
        }
        
        print(f"✅ 完成! 提取到 {len(final_triples)} 个三元组")
        print(f"   刘彻→卫青: {liuche_to_weiqing} 个 ({result['statistics']['liuche_to_weiqing_percentage']:.1f}%)")
        print(f"   卫青→刘彻: {weiqing_to_liuche} 个 ({result['statistics']['weiqing_to_liuche_percentage']:.1f}%)")
        
        return result


class PipelineManager:
    """流水线管理器"""
    
    def __init__(self, config: PipelineConfig):
        self.config = config
        self.triple_extractor = TripleExtractor(config)
        
        # 确保输出目录存在
        config.output_base.mkdir(parents=True, exist_ok=True)
        
        # 创建日志文件
        self.log_file = config.output_base / f"triple_extraction_log_{datetime.now().strftime('%Y%m%d_%H%M%S')}.txt"
        
        # 统计信息列表
        self.statistics_list = []
    
    def log(self, message: str):
        """记录日志"""
        timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        log_message = f"[{timestamp}] {message}"
        
        # 打印到控制台
        print(log_message)
        
        # 写入日志文件
        with open(self.log_file, 'a', encoding='utf-8') as f:
            f.write(log_message + '\n')
    
    def load_works(self) -> List[Dict]:
        """加载作品数据"""
        try:
            with open(self.config.data_path, 'r', encoding='utf-8') as f:
                all_works = json.load(f)
            
            self.log(f"✅ 读取 {len(all_works)} 个作品")
            
            # 限制处理数量
            if self.config.max_works:
                works_to_process = all_works[:self.config.max_works]
                self.log(f"🔬 处理前 {len(works_to_process)} 个作品")
            else:
                works_to_process = all_works
                self.log(f"🔬 处理所有 {len(works_to_process)} 个作品")
            
            return works_to_process
            
        except Exception as e:
            self.log(f"❌ 读取数据失败: {e}")
            return []
    
    def run_triple_extraction(self):
        """运行三元组抽取"""
        self.log("\n" + "="*70)
        self.log("开始运行三元组抽取")
        self.log("="*70)
        
        start_time = time.time()
        
        # 1. 测试API
        if not self.triple_extractor.test_api_connection():
            self.log("❌ API连接测试失败，退出")
            return
        
        # 2. 加载数据
        works = self.load_works()
        if not works:
            self.log("❌ 无作品可处理，退出")
            return
        
        # 3. 处理所有作品
        all_results = []
        self.log(f"\n开始处理 {len(works)} 个作品...")
        
        for i, work in enumerate(works, 1):
            self.log(f"\n📚 处理第 {i}/{len(works)} 个作品: {work.get('work_id')} - {work.get('title', '无标题')}")
            
            result = self.triple_extractor.extract_from_work(work)
            if result:
                # 保存单个作品结果
                output_file = self.triple_extractor.output_dir / f"{work['work_id']}_triples.json"
                with open(output_file, 'w', encoding='utf-8') as f:
                    json.dump(result, f, ensure_ascii=False, indent=2)
                
                self.log(f"💾 保存到: {output_file}")
                all_results.append(result)
                
                # 收集统计信息
                stats = {
                    "work_id": result["work_id"],
                    "title": result["title"],
                    "updated_words": result["updated_words"],
                    "total_triples": result["total_triples"],
                    "liuche_to_weiqing": result["statistics"]["liuche_to_weiqing"],
                    "weiqing_to_liuche": result["statistics"]["weiqing_to_liuche"],
                    "liuche_to_weiqing_percentage": result["statistics"]["liuche_to_weiqing_percentage"],
                    "weiqing_to_liuche_percentage": result["statistics"]["weiqing_to_liuche_percentage"]
                }
                self.statistics_list.append(stats)
            
            if i < len(works):
                self.log(f"⏳ 等待3秒...")
                time.sleep(3)
        
        # 保存汇总
        if all_results:
            summary_file = self.triple_extractor.output_dir / "triples_summary.json"
            with open(summary_file, 'w', encoding='utf-8') as f:
                json.dump(all_results, f, ensure_ascii=False, indent=2)
            
            self.log(f"📁 三元组汇总已保存到: {summary_file}")
            
            # 计算总体统计
            total_triples = sum(r["total_triples"] for r in all_results)
            liuche_to_weiqing = sum(r["statistics"]["liuche_to_weiqing"] for r in all_results)
            weiqing_to_liuche = sum(r["statistics"]["weiqing_to_liuche"] for r in all_results)
            
            self.log(f"\n📊 总体统计:")
            self.log(f"   总作品数: {len(all_results)}")
            self.log(f"   总三元组数: {total_triples}")
            self.log(f"   刘彻→卫青: {liuche_to_weiqing} 个 ({(liuche_to_weiqing/total_triples*100) if total_triples > 0 else 0:.1f}%)")
            self.log(f"   卫青→刘彻: {weiqing_to_liuche} 个 ({(weiqing_to_liuche/total_triples*100) if total_triples > 0 else 0:.1f}%)")
        
        # 保存统计信息到Excel
        if self.statistics_list:
            self.save_statistics_to_excel()
        
        total_time = time.time() - start_time
        self.log(f"\n⏱️  总耗时: {total_time:.2f} 秒")
        self.log(f"📝 日志文件: {self.log_file}")
        self.log("\n🎉 三元组抽取完成!")
    
    def save_statistics_to_excel(self):
        """保存统计信息到Excel文件"""
        try:
            if not self.statistics_list:
                self.log("⚠️  无统计信息可保存")
                return
            
            # 创建DataFrame
            df = pd.DataFrame(self.statistics_list)
            
            # 重新排列列顺序
            columns_order = [
                'work_id', 'title', 'updated_words', 'total_triples',
                'liuche_to_weiqing', 'weiqing_to_liuche',
                'liuche_to_weiqing_percentage', 'weiqing_to_liuche_percentage'
            ]
            df = df[columns_order]
            
            # 重命名列
            df = df.rename(columns={
                'work_id': '作品ID',
                'title': '作品标题',
                'updated_words': '字数统计',
                'total_triples': '总三元组数',
                'liuche_to_weiqing': '刘彻→卫青数量',
                'weiqing_to_liuche': '卫青→刘彻数量',
                'liuche_to_weiqing_percentage': '刘彻→卫青占比(%)',
                'weiqing_to_liuche_percentage': '卫青→刘彻占比(%)'
            })
            
            # 保存到Excel
            excel_file = self.config.output_base / "triple_statistics.xlsx"
            df.to_excel(excel_file, index=False, engine='openpyxl')
            
            self.log(f"📊 统计信息已保存到Excel: {excel_file}")
            
            # 添加总体统计到另一个工作表
            with pd.ExcelWriter(excel_file, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
                total_stats = pd.DataFrame([{
                    '总作品数': len(df),
                    '总三元组数': df['总三元组数'].sum(),
                    '总刘彻→卫青数量': df['刘彻→卫青数量'].sum(),
                    '总卫青→刘彻数量': df['卫青→刘彻数量'].sum(),
                    '平均刘彻→卫青占比(%)': df['刘彻→卫青占比(%)'].mean(),
                    '平均卫青→刘彻占比(%)': df['卫青→刘彻占比(%)'].mean(),
                    '平均字数': df['字数统计'].mean(),
                    '平均每作品三元组数': df['总三元组数'].mean()
                }])
                total_stats.to_excel(writer, sheet_name='总体统计', index=False)
                
            self.log(f"📈 总体统计已添加到Excel文件")
            
        except Exception as e:
            self.log(f"❌ 保存统计信息到Excel失败: {e}")


def main():
    """主函数"""
    print("="*70)
    print("刘彻-卫青同人小说三元组抽取流水线")
    print("="*70)
    
    # 配置
    config = PipelineConfig(
        data_path=Path(r"AO3_data\LiuWei.json"),
        output_base=Path(r"AO3_data"),
        max_works=None,  # None表示处理所有作品，设置为数字则处理前N个
        test_mode=False  # 设置为True只处理前3个作品
    )
    
    if config.test_mode:
        config.max_works = 3
        print("🔬 测试模式：仅处理前3个作品")
    
    # 创建并运行流水线
    pipeline = PipelineManager(config)
    pipeline.run_triple_extraction()


if __name__ == "__main__":
    main()

刘彻-卫青同人小说三元组抽取流水线
[2026-01-15 09:11:01] 
[2026-01-15 09:11:01] 开始运行三元组抽取
[2026-01-15 09:11:01] ======================================================================
测试三元组提取 API连接...
✅ 三元组提取 API连接成功: <think>

</think>

连接成功
[2026-01-15 09:11:13] ✅ 读取 120 个作品
[2026-01-15 09:11:13] 🔬 处理所有 120 个作品
[2026-01-15 09:11:13] 
开始处理 120 个作品...
[2026-01-15 09:11:13] 
📚 处理第 1/120 个作品: 14569767 - 【汉武/彻青说】《甘泉一夜》part2

📚 处理作品 14569767: 【汉武/彻青说】《甘泉一夜》part2
   文本字数: 2921 字符
   处理块 1/6 (长度: 584 字符)
     提取到 3 个三元组
   处理块 2/6 (长度: 518 字符)
     提取到 5 个三元组
   处理块 3/6 (长度: 596 字符)
     提取到 9 个三元组
   处理块 4/6 (长度: 594 字符)
   处理块 5/6 (长度: 567 字符)
     提取到 7 个三元组
   处理块 6/6 (长度: 62 字符)
✅ 完成! 提取到 24 个三元组
   刘彻→卫青: 20 个 (83.3%)
   卫青→刘彻: 4 个 (16.7%)
[2026-01-15 09:12:55] 💾 保存到: C:\Users\yangx\Desktop\格罗宁根岗位\2026\ChineseRPS_data\AO3_data\0115\triples\14569767_triples.json
[2026-01-15 09:12:55] ⏳ 等待3秒...
[2026-01-15 09:12:58] 
📚 处理第 2/120 个作品: 14944169 - 【刘卫赵霍】观棋不语 No Talking While Playing

📚 处理作品 14944169: 【刘卫赵霍】观棋